In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
plt.ioff();
try:
    maindir
except NameError:
    maindir = os.path.dirname(os.getcwd())
for module in os.listdir(maindir + '\\testing_modules'):
    %run -i "{maindir}\\testing_modules\\{module}"
base = os.path.dirname(os.path.dirname(maindir)) + '\\'
plt.ion();
np.set_printoptions(suppress=True)

In [2]:
%matplotlib qt
import skimage.io
from scipy.ndimage import center_of_mass, gaussian_filter, median_filter
from skimage.measure import label, regionprops, find_contours
from skimage import filters
from skimage import restoration
import pandas as pd
from scipy.optimize import curve_fit
from scipy.signal import find_peaks, correlate, convolve2d
from skimage.feature import peak_local_max
from skimage.segmentation import watershed, morphological_geodesic_active_contour, random_walker, expand_labels
import pyFAI
#import pyFAI.azimuthalIntegrator as ai

In [3]:
filedir = '''DOE_SCGSR\\Data\\221128 Beamtime\\maps\\'''
map_filename = '''scan134664-002.tif'''

map_img = skimage.io.imread(base + filedir + map_filename)
flat_field_est = np.median(map_img, axis=(0, 1))
mask_img = np.bool_(skimage.io.imread(maindir + '''\\testing_resources\\refined_merlin_mask.tif'''))
mask_img[mask_img] = np.nan
flat_img = skimage.io.imread(base + filedir + '''merlin_flat_field.tif''')
print(map_img.shape)

(71, 71, 512, 512)


In [4]:
ai = pyFAI.load(maindir + '\\testing_resources\\ZnO_indexing_v3.poni')
ai.energy = 15
tth_resolution = 0.02 # Degrees
chi_resolution = 0.05 # Degrees

res = ai.integrate2d_ng(map_img.reshape(map_img.shape[0] * map_img.shape[1], 512, 512)[0], 1024, 360, unit='2th_deg', mask=~mask_img, flat=flat_img)
tth, chi = res[1], res[2]
tth_num = int(np.abs(np.max(tth) - np.min(tth)) // tth_resolution)
chi_num = int(np.abs(np.max(chi) - np.min(chi)) // chi_resolution)
# The bounds of interpolation should be limited by the intrument resolution AND the original image size. Should not make up too much data

# There is a better way I could do this...
res = ai.integrate2d_ng(map_img.reshape(map_img.shape[0] * map_img.shape[1], 512, 512)[0], tth_num, chi_num, unit='2th_deg', mask=~mask_img, flat=flat_img)
tth, chi = res[1], res[2]
extent = [tth[0], tth[-1], chi[0], chi[-1]]

In [5]:
res[0].shape

(429, 959)

In [10]:
ai.detector.calc_cartesian_positions()

(array([[0.0000275, 0.0000275, 0.0000275, ..., 0.0000275, 0.0000275,
         0.0000275],
        [0.0000825, 0.0000825, 0.0000825, ..., 0.0000825, 0.0000825,
         0.0000825],
        [0.0001375, 0.0001375, 0.0001375, ..., 0.0001375, 0.0001375,
         0.0001375],
        ...,
        [0.0280225, 0.0280225, 0.0280225, ..., 0.0280225, 0.0280225,
         0.0280225],
        [0.0280775, 0.0280775, 0.0280775, ..., 0.0280775, 0.0280775,
         0.0280775],
        [0.0281325, 0.0281325, 0.0281325, ..., 0.0281325, 0.0281325,
         0.0281325]], dtype=float32),
 array([[0.0000275, 0.0000825, 0.0001375, ..., 0.0280225, 0.0280775,
         0.0281325],
        [0.0000275, 0.0000825, 0.0001375, ..., 0.0280225, 0.0280775,
         0.0281325],
        [0.0000275, 0.0000825, 0.0001375, ..., 0.0280225, 0.0280775,
         0.0281325],
        ...,
        [0.0000275, 0.0000825, 0.0001375, ..., 0.0280225, 0.0280775,
         0.0281325],
        [0.0000275, 0.0000825, 0.0001375, ..., 0.0280225,

In [377]:
blob_img[*spots[1].T[::-1]]

# Fitting all blobs with peak functions
blob_df = fit_blobs(fixed_img, sig_thresh, blob_df, LorentzianFunctions(), tth=tth, chi=chi, plotme=False)

# Find spots within blobs
blob_df = find_blob_spots(fixed_img, blob_df, tth=tth, chi=chi, plotme=False)

# Fit spots within blobs
spot_df = adaptive_spot_fitting(fixed_img, blob_df, sig_thresh, LorentzianFunctions(), tth=tth, chi=chi, plotme=False)

Fitting successful for 3 spot(s) within blob 0!
Fitting successful for 2 spot(s) within blob 1!
Fitting successful for 1 spot(s) within blob 2!
Fitting successful for 1 spot(s) within blob 3!


In [378]:
spot_df

,height,tth,chi,tth_width,chi_width,rotation,parent_blobs
0,266.944883,58.891385,-87.526137,0.012445,0.079773,-2.974994,0
1,74.792800,66.510825,-91.983368,0.013550,0.127293,-11.588731,3
2,58.485185,55.809370,-97.997637,0.010739,0.086922,21.182793,2
3,54.032797,59.111501,-86.513540,0.015331,0.025904,-44.278033,0
4,53.788434,59.131540,-87.476018,0.026761,0.245441,-34.460358,0
5,43.148298,62.613761,-89.530898,0.055343,0.212193,-13.818091,1
6,30.936661,62.505686,-89.480541,0.003891,0.258661,-2.940250,1


In [379]:
q_dict = {}
for spot_num in range(len(spot_df)):
    q_arr = np.array([
        -np.sin(np.radians(spot_df['tth'][spot_num])) * np.sin(np.radians(spot_df['chi'][spot_num])),
        -np.sin(np.radians(spot_df['tth'][spot_num])) * np.cos(np.radians(spot_df['chi'][spot_num])),
        1 - np.cos(np.radians(spot_df['tth'][spot_num]))
    ]) / energy_2_wavelength(15)
    q_dict[str(spot_num)] = 2 * np.pi * q_arr

In [386]:
1 / (np.linalg.norm(q_dict['1'] - q_dict['2']) / 2 / np.pi)

3.9797861445949145

In [322]:
np.linalg.norm(q_vect(spot_df['tth'][2], spot_df['chi'][2], energy_2_wavelength(15)) - q_vect(spot_df['tth'][3], spot_df['chi'][3], energy_2_wavelength(15)))

1.354346747130768

In [304]:
stibnite.lattice._parameters

OrderedDict([('a', 11.28),
             ('b', 3.83),
             ('c', 11.2),
             ('alpha', 90),
             ('beta', 90),
             ('gamma', 90)])

In [325]:
1 / (np.linalg.norm(q_dict['0'] - q_dict['5']) / 2 / np.pi)

11.517893295323681

In [301]:
stibnite.planeDistance(2, 0, 1)

5.037352607977438

In [157]:
h, k, l = 0, 0, 1
q = stibnite.B[:, 0] * h + stibnite.B[:, 1] * k + stibnite.B[:, 2] * l
print(q)

[-0.         -0.          0.56099869]


In [228]:
stibnite.planeDistance(0, 1, 0)

3.83

In [248]:
stibnite.Q(1, 0, 0)

array([0.55701997, 0.        , 0.        ])

In [238]:
1 / np.linalg.norm(q_dict['0'] / 2 / np.pi)

0.755239795966538

In [232]:
1 / (stibnite.Q(0, 1, 0) / 2 / np.pi)

array([-6.25486467e+16,  3.83000000e+00,             inf])

In [73]:
q_dict

{'0': array([ 1.10671049, -0.05828785,  0.72456166]),
 '1': array([ 1.10783083, -0.05297045,  0.72651237]),
 '2': array([ 1.03099368, -0.1738082 ,  0.60111555]),
 '3': array([ 1.07104294, -0.11715031,  0.65952685]),
 '4': array([ 1.06983034, -0.12193664,  0.65820945]),
 '5': array([ 1.10606899, -0.07205119,  0.72494759]),
 '6': array([1.14028419, 0.01474869, 0.80581808])}

In [114]:
poss = list(stibnite.lattice.get_allowed_hkl(1.5))

In [153]:
1 / np.linalg.norm(stibnite.Q(0, 0, 1) / 2 / np.pi)

11.2

In [151]:
stibnite.planeDistance(0,0,1)

11.2

In [134]:
stibnite.planeDistance(poss[15])

5.0158867903665305

In [143]:
np.linalg.norm(q_dict['2'] - q_dict['3'])

0.09069696933832895

In [147]:
np.linalg.norm(q_dict['0'] - q_dict['2'])

0.18524828536322982

In [146]:
np.linalg.norm(q_dict['0'] - q_dict['3'])

0.09469154417261984

In [8]:
stibnite = Phase.fromCIF(base + '''Literature\\CIF\\Diffraction Sb2S3 short-b.cif''')
stibnite.name = 'Stibnite'

In [18]:
a1, a2, a3 = stibnite.a1, stibnite.a2, stibnite.a3

In [28]:
print(stibnite.lattice._bi[0, :])
print(stibnite.lattice._bi[1, :])
print(stibnite.lattice._bi[2, :])

[0.55701997 0.         0.        ]
[-0.          1.64051836  0.        ]
[-0.         -0.          0.56099869]


In [41]:
q_guess = stibnite.Q(5, -3, -11)

In [53]:
np.linalg.norm(q_guess)

8.370158212513099

In [54]:
np.linalg.norm(q_guess / np.pi / 2)

1.3321520539826823

In [50]:
stibnite.planeDistance(5, -3, -11)

0.7506650588499558

In [46]:
q_guess / np.pi / 2

array([ 0.44326241, -0.78328982, -0.98214286])

In [52]:
np.linalg.norm([0.915, -0.048, 0.599]) / energy_2_wavelength(15)

1.3243815940678736

In [36]:
stibnite.Q([5, -3, -11])

array([ 2.78509987, -4.92155507, -6.17098557])

In [38]:
stibnite.HKL(2 * np.pi * np.array([0.915, -0.048, 0.599]))

array([10.3212 , -0.18384,  6.7088 ])

In [326]:
data = stibnite.get_hkl_reflections(en=15e3, tth_range=(tth[0], tth[-1]), ignore_less=0)

In [351]:
angle_tolerance = 0.5
check_equivalents = True
spot_pot_hkls = []
for spot_num in range(len(spot_df)):
    pot_hkl = []
    diff_arr = np.abs(spot_df['tth'][spot_num] - data['tth_angles'])
    match_mask = diff_arr < angle_tolerance

    sorted_hkls = [x for _, x in sorted(zip(data['intensities'][match_mask], data['hkl_reflection'][match_mask]))][::-1]
    if check_equivalents:
        for reflection in sorted_hkls:
            pot_hkl.extend(list(stibnite.lattice.equivalent_hkls(reflection)))
    else:
        pot_hkl.extend(sorted_hkls)
    spot_pot_hkls.append(pot_hkl)

# Add potential hkl information to blob dataframe
if 'potential_hkls' not in spot_df:
    spot_df.insert(len(spot_df.columns), 'potential_hkls', spot_pot_hkls, True)
else:
    spot_df['potential_hkls'] = pd.DataFrame.from_dict({'potential_hkls':spot_pot_hkls})

In [352]:
spot_df

,height,tth,chi,tth_width,chi_width,rotation,parent_blobs,potential_hkls
0,266.944883,58.891385,-87.526137,0.012445,0.079773,-2.974994,0,"[(-6, 3, 8), (6, -3, -8), (6, 3, -8), (6, -3, ..."
1,74.792800,66.510825,-91.983368,0.013550,0.127293,-11.588731,3,"[(-5, -3, -11), (5, -3, 11), (-5, 3, -11), (-5..."
2,58.485185,55.809370,-97.997637,0.010739,0.086922,21.182793,2,"[(-7, -3, 6), (7, -3, -6), (-7, 3, 6), (7, -3,..."
3,54.032797,59.111501,-86.513540,0.015331,0.025904,-44.278033,0,"[(-6, 3, 8), (6, -3, -8), (6, 3, -8), (6, -3, ..."
4,53.788434,59.131540,-87.476018,0.026761,0.245441,-34.460358,0,"[(-6, 3, 8), (6, -3, -8), (6, 3, -8), (6, -3, ..."
5,43.148298,62.613761,-89.530898,0.055343,0.212193,-13.818091,1,"[(-6, -4, 5), (6, -4, -5), (-6, 4, 5), (6, 4, ..."
6,30.936661,62.505686,-89.480541,0.003891,0.258661,-2.940250,1,"[(-6, -4, 5), (6, -4, -5), (-6, 4, 5), (6, 4, ..."


In [353]:
index_df = spot_df.copy()
combo_dict = {}

In [354]:
spot1, spot2 = 0, 1

In [355]:
angle_matrix = stibnite.planeAngle(index_df['potential_hkls'][spot1], index_df['potential_hkls'][spot2])

In [346]:
delta_chi = np.abs(index_df['chi'][spot1] - index_df['chi'][spot2])
print(delta_chi)

6.014269600797249


In [347]:
diff_arr = np.abs(delta_chi - angle_matrix)
match_mask = diff_arr < 1.5
np.sum(match_mask)

0

In [348]:
refl2, refl1 = np.where((match_mask))
hkls1 = np.array([index_df['potential_hkls'][spot1][i] for i in refl1])
hkls2 = np.array([index_df['potential_hkls'][spot2][i] for i in refl2])
#hkls1 = np.array([index_df['potential_hkls'][spot1][i] for i in np.unique(refl1)])
#hkls2 = np.array([index_df['potential_hkls'][spot2][i] for i in np.unique(refl2)])

#index_df['potential_hkls'].at[spot1] = hkls1
#index_df['potential_hkls'].at[spot2] = hkls2
combo_dict[str(spot1) + str(spot2)] = np.array([hkls1, hkls2]).transpose(1, 0, 2)

ValueError: axes don't match array

In [343]:
combo_dict['02']

array([[[10,  3,  0],
        [ 9,  3,  2]],

       [[ 2,  4,  6],
        [ 3,  4,  4]],

       [[12,  2,  1],
        [11,  2,  3]],

       [[ 6, -4,  2],
        [ 4, -4,  3]]])

In [ ]:
for i, value in enumerate(combo_dict['02'][:, 0]):
    try:
        match = list(totuple(combo_dict['03'][:, 0])).index(tuple(value))
        print(match)
    except ValueError:
        pass

In [611]:
np.where(combo_dict['03'][:, 0] == val0)

(array([0, 0, 0, 3, 4, 8, 9], dtype=int64),
 array([0, 1, 2, 0, 1, 0, 0], dtype=int64))

In [189]:
def totuple(a):
    if a.shape == ():
        return a.item()
    else:
        return tuple(map(totuple, a))

In [603]:
set02 = set(totuple(combo_dict['02']))
set03 = set(totuple(combo_dict['03']))

In [598]:
list(map(tuple, combo_dict['02'].tolist()))

[([11, 3, 5], [9, 3, 5]),
 ([11, 0, 10], [9, 0, 10]),
 ([1, 3, 12], [2, 3, 10]),
 ([4, 2, -13], [2, 2, -12]),
 ([11, 0, 10], [11, 0, 8]),
 ([11, 3, 5], [10, 3, 3]),
 ([5, -3, -11], [5, -3, -9]),
 ([10, 0, 11], [8, 0, 11])]

In [593]:
list(map(tuple, map(tuple, combo_dict['02'])))

[(array([11,  3,  5]), array([9, 3, 5])),
 (array([11,  0, 10]), array([ 9,  0, 10])),
 (array([ 1,  3, 12]), array([ 2,  3, 10])),
 (array([  4,   2, -13]), array([  2,   2, -12])),
 (array([11,  0, 10]), array([11,  0,  8])),
 (array([11,  3,  5]), array([10,  3,  3])),
 (array([  5,  -3, -11]), array([ 5, -3, -9])),
 (array([10,  0, 11]), array([ 8,  0, 11]))]

In [619]:
matches = []
for i in combo_dict['02']:
    for j in combo_dict['03']:
        if np.all(i[0] == j[0]):
            

[[11  3  5]
 [ 9  3  5]]
[[11  0 10]
 [ 9  0 10]]
[[11  0 10]
 [ 9  0 10]]
[[ 1  3 12]
 [ 2  3 10]]
[[11  0 10]
 [11  0  8]]
[[11  0 10]
 [11  0  8]]
[[11  3  5]
 [10  3  3]]
[[  5  -3 -11]
 [  5  -3  -9]]
[[10  0 11]
 [ 8  0 11]]
[[10  0 11]
 [ 8  0 11]]


In [684]:
def planeAngle(self, hkl1, hkl2):
    a, b, c = list(self.lattice._parameters.values())[:3]
    alpha, beta, gamma = np.radians(list(self.lattice._parameters.values())[3:])

    hkl1 = np.asarray(hkl1)
    hkl2 = np.asarray(hkl2)
    if len(hkl1.shape) == 1:
        hkl1 = hkl1.reshape(1, *hkl1.shape)
    if len(hkl2.shape) == 1:
        hkl2 = hkl2.reshape(1, *hkl2.shape)
    h1, k1, l1 = hkl1.T
    h2, k2, l2 = hkl2.T

    # Useful constants
    V = self.lattice.UnitCellVolume()
    S11 = b**2 * c**2 * np.sin(alpha)**2
    S22 = a**2 * c**2 * np.sin(beta)**2
    S33 = a**2 * b**2 * np.sin(gamma)**2
    S12 = a * b * c**2 * (np.cos(alpha) * np.cos(beta) - np.cos(gamma))
    S23 = a**2 * b * c * (np.cos(beta) * np.cos(gamma) - np.cos(alpha))
    S13 = a * b**2 * c * (np.cos(gamma) * np.cos(alpha) - np.cos(beta))

    d1 = self.planeDistances(hkl1)
    d2 = self.planeDistances(hkl2)

    vec = (S11 * np.outer(h1, h2)
    + S22 * np.outer(k1, k2)
    + S33 * np.outer(l1, l2)
    + S23 * (np.outer(k1, l2) + np.outer(k2, l1).T)
    + S13 * (np.outer(l1, h2) + np.outer(l2, h1).T)
    + S12 * (np.outer(h1, k2) + np.outer(h2, k1).T))

    phi = np.degrees(np.arccos((np.outer(np.outer(d1, d2), vec) / V**2)))

    # Output such that phi[0] will yield planar angles for hkl1[0]
    phi = np.diag(phi).reshape(len(hkl1), len(hkl2)).T
    return phi

In [601]:
spot_df['tth'][0]

diff_arr = np.abs(spot_df['tth'][0] - data['tth_angles'])
match_mask = diff_arr < 0.5

In [605]:
reflections = data['hkl_reflection'][match_mask]

In [606]:
reflections[0]

array([11,  2, -8])

In [610]:
[*stibnite.lattice.equivalent_hkls(reflections[0])]

[(-11, 2, -8),
 (-11, -2, -8),
 (-11, 2, 8),
 (11, 2, 8),
 (11, 2, -8),
 (-11, -2, 8),
 (11, -2, 8),
 (11, -2, -8)]

In [481]:
def planeAngle(hkl1, hkl2):
    a, b, c = list(stibnite.lattice._parameters.values())[:3]
    alpha, beta, gamma = np.radians(list(stibnite.lattice._parameters.values())[3:])
    
    hkl1 = np.asarray(hkl1)
    hkl2 = np.asarray(hkl2)
    if len(hkl1.shape) == 1:
        hkl1 = hkl1.reshape(1, *hkl1.shape)
    if len(hkl2.shape) == 1:
        hkl2 = hkl2.reshape(1, *hkl2.shape)
    h1, k1, l1 = hkl1.T
    h2, k2, l2 = hkl2.T

    # Useful constants
    V = stibnite.lattice.UnitCellVolume()
    S11 = b**2 * c**2 * np.sin(alpha)**2
    S22 = a**2 * c**2 * np.sin(beta)**2
    S33 = a**2 * b**2 * np.sin(gamma)**2
    S12 = a * b * c**2 * (np.cos(alpha) * np.cos(beta) - np.cos(gamma))
    S23 = a**2 * b * c * (np.cos(beta) * np.cos(gamma) - np.cos(alpha))
    S13 = a * b**2 * c * (np.cos(gamma) * np.cos(alpha) - np.cos(beta))

    d1 = stibnite.planeDistances(hkl1)
    d2 = stibnite.planeDistances(hkl2)

    phi = np.arccos(d1 * d2 / V**2
                    * S11 * h1 * h2
                    + S22 * k1 * k2
                    + S33 * l1 * l2
                    + S23 * (k1 * l2 + k2 * l1)
                    + S13 * (l1 * h2 + l2 * h1)
                    + S12 * (h1 * k2 + h2 * k1)
                    )
    return np.degrees(phi)

In [574]:
hkl_lst = [[1, 1, 1], [1, 3, 1], [1, 1, 3]]

In [589]:
a, b, c = list(stibnite.lattice._parameters.values())[:3]
alpha, beta, gamma = np.radians(list(stibnite.lattice._parameters.values())[3:])

hkl1 = np.asarray(hkl_lst[:])
hkl2 = np.asarray(hkl_lst[:2])
if len(hkl1.shape) == 1:
    hkl1 = hkl1.reshape(1, *hkl1.shape)
if len(hkl2.shape) == 1:
    hkl2 = hkl2.reshape(1, *hkl2.shape)
h1, k1, l1 = hkl1.T
h2, k2, l2 = hkl2.T
print(h1[1], k1[1], l1[1])
print(h2[0], k2[0], l2[0])

# Useful constants
V = stibnite.lattice.UnitCellVolume()
S11 = b**2 * c**2 * np.sin(alpha)**2
S22 = a**2 * c**2 * np.sin(beta)**2
S33 = a**2 * b**2 * np.sin(gamma)**2
S12 = a * b * c**2 * (np.cos(alpha) * np.cos(beta) - np.cos(gamma))
S23 = a**2 * b * c * (np.cos(beta) * np.cos(gamma) - np.cos(alpha))
S13 = a * b**2 * c * (np.cos(gamma) * np.cos(alpha) - np.cos(beta))

d1 = stibnite.planeDistances(hkl1)
d2 = stibnite.planeDistances(hkl2)

vec = (S11 * np.outer(h1, h2)
+ S22 * np.outer(k1, k2)
+ S33 * np.outer(l1, l2)
+ S23 * (np.outer(k1, l2) + np.outer(k2, l1).T)
+ S13 * (np.outer(l1, h2) + np.outer(l2, h1).T)
+ S12 * (np.outer(h1, k2) + np.outer(h2, k1).T))

phi = np.degrees(np.arccos((np.outer(np.outer(d1, d2), vec) / V**2)))

# Output such that phi[0] will yield planar angles for hkl1[0]
phi = np.diag(phi).reshape(len(hkl1), len(hkl2)).T

print(phi)

1 3 1
1 1 1
[[        nan 16.60372454 26.22717919]
 [16.60372454  0.         39.21389635]]


In [592]:
phi[0]

array([        nan, 16.60372454, 26.22717919])

In [581]:
stibnite.planeAngle(hkl_lst[1], hkl_lst[2])

39.21389635364892

In [568]:
vec = (S11 * np.outer(h1, h2)
+ S22 * np.outer(k1, k2)
+ S33 * np.outer(l1, l2)
+ S23 * (np.outer(k1, l2) + np.outer(k2, l1))
+ S13 * (np.outer(l1, h2) + np.outer(l2, h1))
+ S12 * (np.outer(h1, k2) + np.outer(h2, k1)))

In [571]:
np.diag(np.degrees(np.arccos((np.outer(np.outer(d1, d2), vec) / V**2)))).reshape(3, 3)

array([[        nan, 16.60372454, 26.22717919],
       [16.60372454,  0.        , 39.21389635],
       [26.22717919, 39.21389635,         nan]])

In [455]:
np.degrees(np.arccos(np.einsum('i,j,k->ijk', d1, d2, vec) / V**2))

array([[[84.6315719 , 35.75340716, 84.55437524],
        [88.04120889, 72.75344271, 88.01311504],
        [85.94263114, 52.14008464, 85.88436217]],

       [[88.04120889, 72.75344271, 88.01311504],
        [89.28450463, 83.78173698, 89.27424622],
        [88.51877221, 77.04307106, 88.49753141]],

       [[85.94263114, 52.14008464, 85.88436217],
        [88.51877221, 77.04307106, 88.49753141],
        [86.93268149, 62.34539175, 86.88866319]]])

In [466]:
stibnite.planeAngle(hkl_lst[0], hkl_lst[2])

26.133720714363392

In [415]:
np.degrees(np.arccos(np.einsum('i,j,k->j', d1, d2, vec) / V**2))

array([nan, nan, nan, nan])

In [355]:
(np.dot(np.outer(d1, d2), vec) / V**2)

array([8.06262198, 2.73757466, 8.00544027, 2.59222505])

In [153]:
data = stibnite.get_hkl_reflections(en=15e3, tth_range=(tth[0], tth[-1]), ignore_less=0)

fig, ax = plt.subplots(1, 1, figsize=(6, 5), dpi=200)
im = ax.imshow(fixed_img, extent=[tth[0], tth[-1], chi[0], chi[-1]], vmin=0)
fig.colorbar(im, ax=ax)
ax.set_xlabel('Scattering Angle, 2θ [°]')
ax.set_ylabel('Azimuthal Angle, χ [°]')
for i in range(len(blob_df)):
    ax.plot(*blob_df['contour'][i], c='r', lw=0.5)
ax.scatter(spot_df['tth'], spot_df['chi'], c='r', s=1)
ax.vlines(data['tth_angles'], ymin=chi[0], ymax=chi[-1], colors='k', lw=0.5, alpha=0.5, linestyles='dashed')
plt.show()

In [5]:
# Integrated blob intensity for the full map.
'''mult = 0.75
sig_thresh = mult * np.std(np.multiply(flat_img, mask_img))

plot_profile = []

for i, img in enumerate(map_img.reshape(map_img.shape[0] * map_img.shape[1], *map_img.shape[2:])):
    test_img = interpolate_merlin_mask(np.multiply((np.copy(img) - flat_img), mask_img))
    fixed_img = find_outlier_pixels(test_img, size=2, tolerance=3, significance=5 * np.std(np.multiply(flat_img, mask_img)))
    proc_img = gaussian_filter(fixed_img, sigma=3)
    plot_profile.append(np.sum(fixed_img[proc_img > sig_thresh]))

plot_profile = np.asarray(plot_profile).reshape(*map_img.shape[:2])'''

In [ ]:
# Determine and plot residual image, within found blobs...
'''fit_img = np.zeros_like(fixed_img)

for i in range(len(blob_df)):
    if not np.any(np.isnan(blob_df['gaussian_fit'][i])):
        blob_fit = gaussian_2d((x_coords, y_coords), *blob_df['gaussian_fit'][i][:-1]) - blob_df['gaussian_fit'][i][3]
        fit_img += blob_fit

fit_img += np.mean(np.asarray([x for x in blob_df['gaussian_fit'].to_list() if np.any(~np.isnan(x))])[:, 3])
res_img = fixed_img * ~bkg_mask - fit_img * ~bkg_mask

fig, ax = plt.subplots(1, 3, figsize=(10, 3), dpi=200, sharex=True, sharey=True)
im = ax[0].imshow(fixed_img, vmin=0, vmax=100)
fig.colorbar(im , ax=ax[0], shrink=0.6)
ax[0].set_title('Experiment')
im = ax[1].imshow(fit_img, vmin=0, vmax=100)
fig.colorbar(im , ax=ax[1], shrink=0.6)
ax[1].set_title('Blob Fit')
im = ax[2].imshow(res_img, vmin=-50, vmax=50, cmap='bwr')
fig.colorbar(im , ax=ax[2], shrink=0.6)
ax[2].set_title('Residual')

for i in range(len(blob_df)):
    ax[0].plot(blob_df['contour'][i].T[1], blob_df['contour'][i].T[0], c='r', lw=0.5)
    ax[1].plot(blob_df['contour'][i].T[1], blob_df['contour'][i].T[0], c='r', lw=0.5)
    ax[2].plot(blob_df['contour'][i].T[1], blob_df['contour'][i].T[0], c='r', lw=0.5)

plt.show()'''

In [15]:
'''blob_num = 1

#filt_img = median_filter(fixed_img * blob_df['mask'][1], size=2)
filt_img = gaussian_filter(median_filter(fixed_img * blob_df['mask'][blob_num], size=2), sigma=0.5)
#filt_img = -filters.sobel(gaussian_filter(fixed_img * blob_df['mask'][1], sigma=1.5))
#filt_img = filters.roberts(filters.roberts(gaussian_filter(fixed_img * blob_df['mask'][1], sigma=1)))

x_img_lst = np.linspace(0, tth_num - 1, tth_num)
y_img_lst = np.linspace(0, chi_num - 1, chi_num)
x_img, y_img = np.meshgrid(x_img_lst, y_img_lst)

blob_bbox = [int(np.min(x_img[blob_df['mask'][blob_num]])), int(np.max(x_img[blob_df['mask'][blob_num]])) + 1,
             int(np.min(y_img[blob_df['mask'][blob_num]])), int(np.max(y_img[blob_df['mask'][blob_num]])) + 1]

fig, ax = plt.subplots(1, 2, figsize=(6, 5), dpi=200, sharex=True, sharey=True)
ax[0].imshow(fixed_img, extent=extent, vmin=0)
ax[1].imshow(filt_img, extent=extent, vmin=0)
# Peak search in blobl bounding box to reduce computational time
local_spots = peak_local_max(filt_img[blob_bbox[2]:blob_bbox[3], blob_bbox[0]:blob_bbox[1]], min_distance=3, threshold_rel=0.15)
if local_spots.shape[0] == 0:
    print('No local maxima found. Using total blob maximum.')
    local_spots = np.array([blob_df['max_coords'][blob_num]])[0]
    local_max = np.max(blob_df['value'][blob_num])
elif local_spots.shape[0] > 1: # discounts any insignificant peaks if more than one local maxima found
    significant_spots = [filt_img[blob_bbox[2]:blob_bbox[3], blob_bbox[0]:blob_bbox[1]][*spot] > np.std(fixed_img) for spot in local_spots]
    local_spots = local_spots[significant_spots]
local_max = [fixed_img[blob_bbox[2]:blob_bbox[3], blob_bbox[0]:blob_bbox[1]][*spot] for spot in local_spots]
local_spots = np.array([[x_coords[blob_bbox[2]:blob_bbox[3], blob_bbox[0]:blob_bbox[1]][*spot],
                y_coords[blob_bbox[2]:blob_bbox[3], blob_bbox[0]:blob_bbox[1]][*spot]] for spot in local_spots])
for axi in ax:
    axi.scatter(local_spots[:, 0], local_spots[:, 1], c='r', s=1)
    axi.plot(*blob_df['contour'][blob_num], c='r', lw=0.5)

plt.show()
print(f'Found {len(local_spots)} possible spots within blob {blob_num}.')'''

Found 2 possible spots within blob 1.


In [133]:
# Alternate method for finding local maxima
'''filt_img = -filters.sobel(gaussian_filter(fixed_img * blob_df['mask'][1], sigma=1))

local_maxima = []
for i, j in zip(y_coords[blob_bbox[2]:blob_bbox[3], blob_bbox[0]:blob_bbox[1]].flatten(), x_coords[blob_bbox[2]:blob_bbox[3], blob_bbox[0]:blob_bbox[1]].flatten()):
    i, j = int(i), int(j)
    condition = np.all([(filt_img[i, j] > filt_img[i + 1, j]),
                        (filt_img[i, j] > filt_img[i - 1, j]),
                        (filt_img[i, j] > filt_img[i, j + 1]),
                        (filt_img[i, j] > filt_img[i, j - 1])])
    if condition:
        local_maxima.append([i, j])
local_maxima = np.asarray(local_maxima)

fig, ax = plt.subplots(1, 2, figsize=(6, 5), dpi=200, sharex=True, sharey=True)
ax[0].imshow(fixed_img, vmin=0, vmax=100)
ax[1].imshow(filt_img)
for axi in ax:
    axi.scatter(x_coords[local_maxima[:, 0], local_maxima[:, 1]], y_coords[local_maxima[:, 0], local_maxima[:, 1]], c='r', s=1)

plt.show()''';

In [17]:
# Determine and plot residual image, within found blobs...
'''fit_img = np.zeros_like(fixed_img)
contours = find_contours(blob_img, 1)

fit_img = PeakModel.multi_2d((x_coords.ravel(), y_coords.ravel()), *popt).reshape(*fixed_img.shape)

res_img = fixed_img * ~bkg_mask - fit_img * ~bkg_mask

fig, ax = plt.subplots(1, 3, figsize=(10, 3), dpi=200, sharex=True, sharey=True)
im = ax[0].imshow(fixed_img, extent=extent, vmin=0, vmax=100)
fig.colorbar(im, ax=ax[0])
ax[0].set_title('Experiment')

im = ax[1].imshow(fit_img, extent=extent, vmin=0, vmax=100)
fig.colorbar(im, ax=ax[1])
ax[1].set_title('Fit')

im = ax[2].imshow(res_img, extent=extent, vmin=-100, vmax=100, cmap='bwr', interpolation=None)
fig.colorbar(im, ax=ax[2])
ax[2].set_title('Residual')

for axi in ax:
    axi.plot(*blob_df['contour'][blob_num], c='r', lw=0.5)

plt.show()'''

In [642]:
correlations = []
for blob_num in range(len(blob_df)):    
    # Cross-correlation of blobs
    blob_bbox = [int(np.min(x_coords[blob_df['mask'][blob_num]])), int(np.max(x_coords[blob_df['mask'][blob_num]])) + 1,
                int(np.min(y_coords[blob_df['mask'][blob_num]])), int(np.max(y_coords[blob_df['mask'][blob_num]])) + 1]
    z_fit = blob_df['value'][blob_num][blob_bbox[2]:blob_bbox[3], blob_bbox[0]:blob_bbox[1]]
    #x_corr = correlate(fixed_img, z_fit, mode='same')
    x_corr = convolve2d(fixed_img, z_fit, mode='same')
    self_similar = peak_local_max(x_corr, min_distance=1, threshold_rel=0.65)

    for region in self_similar:
        for i, blob in enumerate(blob_df['mask']):
            if blob[*region]:
                correlations.append([blob_num, i])
                #print(f'Blob {blob_num} correlates with blob {i}')

# Probably a better way of doing this than a bunch of one line for loops...
uniq_corr, match_corr = [], []
[uniq_corr.append(x) for x in correlations if x not in uniq_corr and x[0] != x[1]];
[match_corr.append(x) for x in uniq_corr if x not in match_corr and x[::-1] not in match_corr and x[::-1] in uniq_corr];
[print(f'Blobs {x[0]} and {x[1]} might be from the same grain.') for x in match_corr];

In [643]:
blob_num = 1
# Cross-correlation of blobs
blob_bbox = [int(np.min(x_coords[blob_df['mask'][blob_num]])), int(np.max(x_coords[blob_df['mask'][blob_num]])) + 1,
             int(np.min(y_coords[blob_df['mask'][blob_num]])), int(np.max(y_coords[blob_df['mask'][blob_num]])) + 1]
z_fit = blob_df['value'][blob_num][blob_bbox[2]:blob_bbox[3], blob_bbox[0]:blob_bbox[1]]
x_corr = correlate(fixed_img, z_fit, mode='same')
#x_corr = convolve2d(fixed_img, z_fit, mode='same')
self_similar = peak_local_max(x_corr, min_distance=1, threshold_rel=0.65)

fig, ax = plt.subplots(1, 2, figsize=(10, 5), dpi=200, sharex=True, sharey=True)
ax[0].imshow(fixed_img)
ax[0].plot(blob_df['contour'][blob_num].T[1], blob_df['contour'][blob_num].T[0], c='r', lw=0.5)
for region in self_similar:
    ax[0].scatter(region[1], region[0], s=1, c='r')
ax[1].imshow(x_corr)
plt.show()

In [241]:
from skimage.feature import SIFT, match_descriptors

detector_extractor1 = SIFT()
detector_extractor2 = SIFT()
detector_extractor1.detect_and_extract(fixed_img)
detector_extractor2.detect_and_extract(z_fit)
matches = match_descriptors(detector_extractor1.descriptors,
                            detector_extractor2.descriptors,
                            max_ratio=np.inf)
print(matches)

[[262   0]
 [264   1]
 [268   2]
 [269   3]
 [272   6]
 [275   7]
 [279   9]
 [280   8]
 [572  11]
 [575  12]
 [783  13]
 [913  15]
 [950  14]]


In [242]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)

ax.imshow(fixed_img, vmin=0)
points = detector_extractor1.keypoints[matches[:, 0]]
ax.scatter(points[:, 1], points[:, 0], s=1, c='r')
ax.plot(blob_df['contour'][blob_num].T[1], blob_df['contour'][blob_num].T[0], c='r', lw=0.5)
plt.show()

In [672]:
print(p0)
print(([0, 45, 45, 0, 0, -45, 0, 45, 25, 0, 0, -45,], [np.inf, 55, 55, np.inf, np.inf, 45, np.inf, 55, 35, np.inf, np.inf, 45,]))
print(generate_bounds(p0, Gaussian.func_2d))

[50, 50, 50, 5, 5, 0, 50, 50, 30, 5, 5, 0]
([0, 45, 45, 0, 0, -45, 0, 45, 25, 0, 0, -45], [inf, 55, 55, inf, inf, 45, inf, 55, 35, inf, inf, 45])
([0, 45.0, 45.0, 0, 0, -45, 0, 45.0, 45.0, 0, 0, -45], [inf, 55.0, 55.0, inf, inf, 45, inf, 55.0, 55.0, inf, inf, 45])


In [680]:
x_lst = np.linspace(0, 100, 100)
x_fit, y_fit = np.meshgrid(x_lst, x_lst)
x_fit = x_fit.ravel()
y_fit = y_fit.ravel()

#z_fit = Gaussian.func_2d((x_fit, y_fit), 50, 50, 50, 5, 5, 0)
p0 = [50, 50, 50, 5, 5, 0, 50, 50, 30, 5, 5, 0]
bounds = ([0, 45, 45, 0, 0, -45, 0, 45, 25, 0, 0, -45,], [np.inf, 55, 55, np.inf, np.inf, 45, np.inf, 55, 35, np.inf, np.inf, 45,])
bounds = generate_bounds(p0, Gaussian.func_2d)
z_fit = Gaussian.multi_2d((x_fit, y_fit), *p0) + (np.random.random(len(x_fit)) - 0.5) * 100
z_filt = gaussian_filter(z_fit, sigma=2)

fig, ax = plt.subplots(1, 2, figsize=(10, 5), dpi=200, sharex=True, sharey=True)
ax[0].imshow(z_fit.reshape((len(x_lst), len(x_lst))))
popt, pcov = curve_fit(Gaussian.multi_2d, (x_fit, y_fit), z_fit, p0=p0 + 5 * np.random.random(len(p0)), bounds=bounds)
z_fit = Gaussian.multi_2d((x_fit, y_fit), *popt)
ax[1].imshow(z_fit.reshape((len(x_lst), len(x_lst))))
plt.show()
print(*popt)

49.4821879058145 49.77467393135979 49.75974296037669 5.36264757762948 4.350332997984219 -20.89152706296693 41.15102361797082 50.4141744637802 30.136968607160178 5.74924925559718 6.50140961698663 25.008254047756036


In [641]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)
z_filt = gaussian_filter(z_fit.reshape((len(x_lst), len(x_lst))), sigma=5)
local_spots = peak_local_max(z_filt, min_distance=3, threshold_rel=0.65)
ax.imshow(z_filt)
ax.scatter(local_spots[:, 1], local_spots[:, 0], c='r', s=1)
plt.show()

In [678]:
generate_bounds([50, 30, 50, 5, 5, 0], Gaussian.func_2d)

([0, 25.0, 45.0, 0, 0, -45], [inf, 35.0, 55.0, inf, inf, 45])

In [438]:
zno_data = pd.read_csv(base + filedir + 'ZnO Integration Plot3.csv', delimiter=';')
x = zno_data['2th_deg'].tolist()
y = zno_data['intensity'].tolist()
background = restoration.rolling_ball(y, radius=1500)

fit_mask = (np.array(x) < 74.5) & (np.array(x) > 56)
x_data = np.array(x)[fit_mask]
y_data = np.array(y - background)[fit_mask]

x_fit = np.linspace(np.min(x_data), np.max(x_data), 5000)
    
# Normalize data
norm_int = 100 * (y_data - np.min(y_data)) / (np.max(y_data) - np.min(y_data))

# Find peak indices
peaks = find_peaks(norm_int, prominence=2, height=1, width=5)[0]
    
p_guess = []
for peak in peaks:
    p_guess.append(y_data[peak]) # a: amplitudes
    p_guess.append(x_data[peak]) # x0: peak centers
    p_guess.append(0.1) # sigma: peak width
    
#popt, pcov = curve_fit(all_peaks_fitting, x_data, y_data, p0=p_guess)
popt, pcov = curve_fit(Gaussian.multi_1d, x_data, y_data, p0=p_guess)


fig, ax = plt.subplots(2, 1, figsize=(9, 7), dpi=200, gridspec_kw={'height_ratios': [5, 1]})

ax[0].plot(x_fit, Gaussian.multi_1d(x_fit, *p_guess), '--', c='gray', label='Guess', lw=0.5)
ax[0].scatter(x_data, y_data, s=1, c='k', label='experiment')
ax[0].plot(x_fit, Gaussian.multi_1d(x_fit, *popt), c='r', label='Fit', lw=1)

ax[0].legend()

residuals =  y_data - Gaussian.multi_1d(x_data, *popt)
ss_res = np.sum(residuals ** 2)
ss_tot = np.sum(((y_data) - np.mean(y_data)) ** 2)
r_squared = 1 - (ss_res / ss_tot)

ax[1].set_title('Residuals')
ax[1].plot(x_data, residuals, c='r', lw=1)
ax[1].set_ylim(-1.15 * np.max(np.abs(residuals)), 1.15 * np.max(np.abs(residuals)))

plt.tight_layout()
print(f'R-squared is {r_squared:.4f}')

(1986,)
1986
('amp', 'x0', 'sigma')
37
dict_values([101.08437782167869, 57.20397, 0.1])
dict_values([41.88960107282807, 59.18836, 0.1])
dict_values([104.64633046617723, 59.72871, 0.1])
dict_values([9.929569739086727, 61.21001, 0.1])
dict_values([79.27510777407757, 62.47704, 0.1])
dict_values([85.56256245487863, 64.88066, 0.1])
dict_values([51.59800245487863, 66.85574, 0.1])
dict_values([5.533499739086729, 68.16934, 0.1])
dict_values([26.22182782167869, 71.11332, 0.1])
dict_values([2.907126111720121, 71.84931, 0.1])
dict_values([3.6235363009666024, 72.45487, 0.1])
dict_values([20.440600466177223, 73.64737, 0.1])
(1986,)
1986
('amp', 'x0', 'sigma')
37
dict_values([101.0843793279533, 57.20397, 0.1])
dict_values([41.88960107282807, 59.18836, 0.1])
dict_values([104.64633046617723, 59.72871, 0.1])
dict_values([9.929569739086727, 61.21001, 0.1])
dict_values([79.27510777407757, 62.47704, 0.1])
dict_values([85.56256245487863, 64.88066, 0.1])
dict_values([51.59800245487863, 66.85574, 0.1])
dict_

In [103]:
from math import sqrt
from skimage import data
from skimage.feature import blob_dog, blob_log, blob_doh
from skimage.color import rgb2gray

import matplotlib.pyplot as plt


image = test_img
image_gray = 1-test_img

blobs_log = blob_log(image_gray, max_sigma=30, num_sigma=10, threshold=.1)

# Compute radii in the 3rd column.
blobs_log[:, 2] = blobs_log[:, 2] * sqrt(2)

blobs_dog = blob_dog(image_gray, max_sigma=30, threshold=.1)
blobs_dog[:, 2] = blobs_dog[:, 2] * sqrt(2)

blobs_doh = blob_doh(image_gray, max_sigma=30, threshold=.01)

blobs_list = [blobs_log, blobs_dog, blobs_doh]
colors = ['yellow', 'lime', 'red']
titles = ['Laplacian of Gaussian', 'Difference of Gaussian',
          'Determinant of Hessian']
sequence = zip(blobs_list, colors, titles)

fig, axes = plt.subplots(1, 3, figsize=(9, 3), sharex=True, sharey=True)
ax = axes.ravel()

for idx, (blobs, color, title) in enumerate(sequence):
    ax[idx].set_title(title)
    ax[idx].imshow(image)
    for blob in blobs:
        y, x, r = blob
        c = plt.Circle((x, y), r, color=color, linewidth=2, fill=False)
        ax[idx].add_patch(c)
    ax[idx].set_axis_off()

plt.tight_layout()
plt.show()